In [1]:
import requests
import mysql.connector
from bs4 import BeautifulSoup
from datetime import datetime
import re  


URL = "https://www.galleriesnow.net/exhibitions/europe/"
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"} 

# Here the user agent is for Edge browser on windows 10. You can find your browser user agent from the above given link. 
r = requests.get(url=URL, headers=headers) 
soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib 
url_containers = soup.find('ul', class_='contentul') #image containers 

#DB connection
conn = mysql.connector.connect(host='127.0.0.1', user = 'root', password = 'Color2023@', db = 'blur_db', port='3310')
cursor = conn.cursor()

# Get exisiting urls 
query = "SELECT url FROM exhibitions"
cursor.execute(query)
conn.close()
url_list = [row[0] for row in cursor.fetchall()]

#Extract new urls to update
urls = []
for img_container in url_containers.find_all('a', class_ = 'glinks show-pic show-pic-wrapper'):
    href = img_container.get('href')
    if href and re.match(r'^https://www.galleriesnow.net/shows/', href):
        if href not in url_list:
            url = {}
            url['href'] = href
            url['img'] = img_container.find('img', class_='show-pic-img').get('data-src')
            urls.append(url)

print(urls)   
print("Number of URL:", len(urls))

[{'href': 'https://www.galleriesnow.net/shows/nick-veasey-forensic-beauty/', 'img': 'https://www.galleriesnow.net/wp-content/uploads/2023/11/Resize-image-project-1-600x450.png'}, {'href': 'https://www.galleriesnow.net/shows/gunter-umberg-zeit-bild/', 'img': 'https://www.galleriesnow.net/wp-content/uploads/2023/11/01_Gunter-Umberg_A-arte-Invernizzi-600x800.jpg'}, {'href': 'https://www.galleriesnow.net/shows/fadia-haddad/', 'img': 'https://www.galleriesnow.net/wp-content/uploads/2023/11/Ab-Anbar-Fadia-Haddad-1-600x400.jpg'}, {'href': 'https://www.galleriesnow.net/shows/alexandre-canonico-still/', 'img': 'https://www.galleriesnow.net/wp-content/uploads/2023/11/Ab-Anbar-Alexandre-Canonico-1-600x400.jpg'}, {'href': 'https://www.galleriesnow.net/shows/merikokeb-berhanu/', 'img': 'https://www.galleriesnow.net/wp-content/uploads/2023/12/Installation-Photography_Merikokeb-Berhanu_A-World-Grows-Within_2023_HighRes_2-600x413.jpg'}, {'href': 'https://www.galleriesnow.net/shows/andreas-breunig/', '

In [8]:
conn = mysql.connector.connect(host='127.0.0.1', user = 'root', password = 'Color2023@', db = 'blur_db', port='3310')
cur = conn.cursor()



In [9]:
import os
from openai import OpenAI


data_inserted = 0

for url in urls:
    try:
        suburl = url['href']
        sub_r = requests.get(url=suburl, headers=headers) 
        sub_soup = BeautifulSoup(sub_r.content, 'html5lib')

        exhibition = []  # a list to store quotes 

        basic_info = sub_soup.find('div', class_= 'buffer-default')

        exhibition = {}
        exhibition['url'] = url['href']
        exhibition['title'] = basic_info.find('h1', attrs = {'id':'exhibition-more'}).text
        exhibition['location'] = basic_info.find('a', class_ = 'link-default').find('span', class_ = 'grey-text').text
        
        museum_text = basic_info.find('div', class_ = 'clearfix-text').find('a', class_ = 'link-default').text
        last_comma_index = museum_text.rfind(',')
        exhibition['museum'] = museum_text[:last_comma_index].strip()
        exhibition['description'] = sub_soup.find('p', attrs = {'id':'content-text'}).text
        exhibition['address'] = sub_soup.find('p', class_ = 'show-space-address').find('a').text
        
        
        client = OpenAI(
        api_key=os.environ["OPENAI_API_KEY"]
        )

        message = "Extract 3 keywords of the text.Put the keywords in the same line, seperating by comma:" + exhibition['description']

        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": message,
                }
            ],
            model="gpt-3.5-turbo",
        )

        reply = chat_completion.choices[0].message.content
        exhibition['tags'] = reply

        #image
        exhibition['thumbnail_image'] = url['img']
        # image_div = sub_soup.find('div', class_= 'banner_fullimg')
        # if image_div is not None:
        #     style_attribute = image_div.get('style')
        #     url_start = style_attribute.find('(') + 1
        #     url_end = style_attribute.find(')')
        #     image_url = style_attribute[url_start:url_end].strip()
        #     exhibition['thumbnail_image'] = image_url
        # else:  exhibition['thumbnail_image'] = ""

        #start date and end date
        date = basic_info.find('div', class_='buffer-default').findAll('p')[0].text

        if 'to' in date:
            start_date = date.split(' to ')[0]
            end_date = date.split(' to ')[1]
        elif '-' in date:
            start_date = date.split(' - ')[0]
            end_date = date.split(' - ')[1]

        current_format = "%a %d %b %Y"
        new_format = "%Y-%m-%d"

        start_date_object = datetime.strptime(start_date, current_format)
        end_date_object = datetime.strptime(end_date, current_format)

        exhibition['start_date'] = start_date_object.strftime(new_format)
        exhibition['end_date'] = end_date_object.strftime(new_format)

        #default
        exhibition['user_id'] = 1
        exhibition['type_id'] = 1
        exhibition['views'] = 0
        exhibition['created_at'] = datetime.now()
        exhibition['updated_at'] = datetime.now()
        
        
        sql = "INSERT INTO exhibitions (user_id, type_id, title, description, location, start_date, end_date, views, thumbnail_image, created_at, updated_at, url, museum, address, tags) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val = (exhibition['user_id'], exhibition['type_id'], exhibition['title'], exhibition['description'], exhibition['location'], exhibition['start_date'], exhibition['end_date'], exhibition['views'], exhibition['thumbnail_image'], exhibition['created_at'], exhibition['updated_at'], exhibition['url'], exhibition['museum'], exhibition['address'], exhibition['tags'])

        cur.execute(sql, val)

        conn.commit()

        data_inserted = data_inserted + 1
    except Exception as e:
        print(f"error!! suburl: {suburl}, Message: {e}")

conn.close()
print(f"{data_inserted} data inserted")

에러 발생! suburl: https://www.galleriesnow.net/shows/franz-west-8/, 에러 메시지: list index out of range


KeyboardInterrupt: 

1 record inserted.
